<a href="https://colab.research.google.com/github/jpantojac/dMet/blob/main/modelamiento/dMet_modelamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bienvenido al Diplomado de Meteorologia**


---




En este espacio se realizaran las entregas de los ejercicios propuestos


In [1]:
import os

print("Bienvenido")
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Bienvenido


MessageError: ignored